<a href="https://colab.research.google.com/github/pedrobcavalcante/Backtest-trading-strategies/blob/main/Gerador_de_Par%C3%A2metros_para_estrat%C3%A9gias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
import backtesting
import yfinance as yf
from backtesting import Backtest, Strategy
backtesting.set_bokeh_output(notebook=False)


In [ ]:
from backtesting.test import SMA


In [ ]:
class HiLo(Strategy):
    n1 = 1
    n2 = 1

    def init(self):
        self.sma1 = self.I(SMA, self.data.High, self.n1)
        self.sma2 = self.I(SMA, self.data.Low, self.n2)

    def next(self):
        if self.data.Close > self.sma1 and not self.position.is_long:
            self.buy()

        if self.sma2 > self.data.Close:
            self.position.close()

    def setInitialParams(self, valor1, valor2):
        self.n1 = valor1
        self.n2 = valor2

In [ ]:
def gerarParametros(dataframe, myStrategy, tamanho):
    maiorRetorno = {}
    BoasEstratégiasParaSalvar = []

    for s1 in range(tamanho):
        s1 += 1
        print("\nTESTE sma1: ", s1,end=' ')
        print("sma2: ", s1, end=" ")
        for s2 in range(tamanho):
            s2 += 1
            if(s2%10 == 0 or s2 == tamanho or s2 == 1 ):
                print("\n         ", end='')
            print("sma2", s2, end=" || ")
            try:
                HiLo.setInitialParams(HiLo, s1, s2)
                bt = Backtest(dataframe, myStrategy,cash=100000000, exclusive_orders=True)
                output = bt.run()
                if maiorRetorno == {}:
                    maiorRetorno = {"sma1": s1, "sma2": s2, "output": output}
                if output['Return [%]'] > maiorRetorno["output"]['Return [%]'] and output['# Trades'] > 9:
                    maiorRetorno = {"sma1": s1, "sma2": s2, "output": output}
                if output['Return [%]'] > 0 and output['# Trades'] > 9:
                    BoasEstratégiasParaSalvar.append(
                        {"sma1": s1, "sma2": s2, "output": output})
            except Exception as e:
                print(e)
    return {"maiorRetorno": maiorRetorno, "BoasEstratégiasParaSalvar": BoasEstratégiasParaSalvar}


Mudar apenas os Parametros dessa celula:

In [ ]:
# perido dos dados a serem textados:
period = '10y'
# tempo dos candlesticks:
interval = '1wk' # 1 semana
# ativo:
ticket = "ITSA4.SA"
# periodo da média móvel:
sma = 20
dataframe = yf.Ticker(ticket).history(period=period, interval=interval)
dataframe = dataframe.dropna()
res = gerarParametros(dataframe, HiLo, sma)


In [ ]:
HiLo.setInitialParams(HiLo, res['maiorRetorno']['sma1'], res['maiorRetorno']['sma2'])
bt = Backtest(dataframe, HiLo,cash=1000000000, exclusive_orders=True)
output = bt.run()
print(output)
bt.plot()